In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
turnovers = ['THROWAWAY', 'DROP', 'STALL', 'CALLAHAN_THROWN']
takeaways = ['BLOCK', 'CALLAHAN', 'STALL_CAUSED', 'THROWAWAY_CAUSED']

pulls = ['PULL_OUT_OF_BOUNDS', 'PULL_INBOUNDS']

pens = ['D_PENALTY_ON_THEM', 'D_PENALTY_ON_US', 'O_PENALTY_ON_THEM', 'O_PENALTY_ON_US']

quarter_ends = ['START_OF_GAME', 'END_OF_Q1', 'HALFTIME', 'END_OF_Q3', 'GAME_OVER']
line_sets = ['SET_D_LINE', 'SET_O_LINE', 'SET_D_LINE_NO_PULL', 'SET_O_LINE_NO_PULL']
timeouts = ['THEIR_MIDPOINT_TIMEOUT', 'OUR_MIDPOINT_TIMEOUT']

other = ['REF_TIMEOUT_DISCUSSION???', 'INJURY_ON_O', 'INJURY_ON_D']


In [3]:
df = pd.read_csv("FlyersWeek3.csv").rename({"Unnamed: 0" : "event_index"}, axis=1)

In [4]:
df.dtypes

event_index          int64
game_id              int64
date                object
home_team            int64
away_team            int64
event_counter        int64
team_id              int64
current_quarter      int64
time                object
event_type          object
player              object
x                  float64
y                  float64
o_point               bool
d_point               bool
dtype: object

In [5]:
list(df.player[1].split(", "))

df['line'] = np.where(df['event_type'].isin(line_sets), list(df['player'].str.split(", ")), "")
df['line']

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


0                                                        
1       [Grayson Sanner, David Richardson, Justin Alle...
2                                                        
3                                                        
4       [Alex Davis, Jacob Fairfax, Matt Gouchoe-Hanas...
                              ...                        
2559                                                     
2560                                                     
2561                                                     
2562                                                     
2563                                                     
Name: line, Length: 2564, dtype: object

In [6]:
df['event_type'].value_counts()
df['event_type'].unique()
df['event_type'].unique()

array(['START_OF_GAME', 'SET_D_LINE', 'PULL_INBOUNDS', 'SCORED_ON',
       'SET_O_LINE', 'POSSESSION', 'D_PENALTY_ON_THEM', 'GOAL',
       'D_PENALTY_ON_US', 'THROWAWAY_CAUSED', 'THROWAWAY',
       'THEIR_MIDPOINT_TIMEOUT', 'SET_D_LINE_NO_PULL', nan,
       'O_PENALTY_ON_THEM', 'BLOCK', 'END_OF_Q1',
       'REF_TIMEOUT_DISCUSSION???', 'O_PENALTY_ON_US',
       'OUR_MIDPOINT_TIMEOUT', 'SET_O_LINE_NO_PULL', 'HALFTIME',
       'END_OF_Q3', 'DROP', 'GAME_OVER', 'PULL_OUT_OF_BOUNDS', 'STALL',
       'INJURY_ON_O', 'INJURY_ON_D', 'STALL_CAUSED', 'CALLAHAN',
       'CALLAHAN_THROWN'], dtype=object)

In [7]:
df.loc[df['event_type'] == "CALLAHAN"]
df.loc[df['event_type'] == "CALLAHAN_THROWN"]
#df.iloc[110:120]
# df.iloc[2360:2372]

,event_index,game_id,date,home_team,away_team,event_counter,team_id,current_quarter,time,event_type,player,x,y,o_point,d_point,line
2366,250,2893,2022-04-29,242,241,250,241,3,Q3 06:46,CALLAHAN_THROWN,NaN,-1.97,16.89,False,True,


In [8]:
df['reciever'] = df['player'].shift(-1)
df['turn'] = np.where(df['event_type'].isin(turnovers), 1, 0)

In [9]:
df['turn'] = np.where(df['event_type'].isin(turnovers), 1, 0)
df['turn'] = np.where(df['event_type'] == "DROP", 0.5, df['turn'])
df['who_turned'] = df['turn'].shift(-1) # This isn't quite right
df['turn'] = np.where(df['event_type'].isin(turnovers), 1, df['turn'])
df

,event_index,game_id,date,home_team,away_team,event_counter,team_id,current_quarter,time,event_type,player,x,y,o_point,d_point,line,reciever,turn,who_turned
0,0,2894,2022-05-14,242,245,0,242,1,NaN,START_OF_GAME,NaN,NaN,NaN,False,False,,"Grayson Sanner, David Richardson, Justin Allen...",0.0,0.0
1,1,2894,2022-05-14,242,245,1,242,1,NaN,SET_D_LINE,"Grayson Sanner, David Richardson, Justin Allen...",NaN,NaN,False,True,"[Grayson Sanner, David Richardson, Justin Alle...",Rick Hennighausen,0.0,0.0
2,2,2894,2022-05-14,242,245,2,242,1,NaN,PULL_INBOUNDS,Rick Hennighausen,-6.89,91.56,False,True,,NaN,0.0,0.0
3,3,2894,2022-05-14,242,245,3,242,1,Q1 11:13,SCORED_ON,NaN,NaN,NaN,False,True,,"Alex Davis, Jacob Fairfax, Matt Gouchoe-Hanas,...",0.0,0.0
4,4,2894,2022-05-14,242,245,4,242,1,NaN,SET_O_LINE,"Alex Davis, Jacob Fairfax, Matt Gouchoe-Hanas,...",NaN,NaN,True,False,"[Alex Davis, Jacob Fairfax, Matt Gouchoe-Hanas...",Matt Gouchoe-Hanas,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2559,443,2893,2022-04-29,242,241,443,241,4,NaN,POSSESSION,Khalif El-Salaam,-20.04,37.20,True,False,,NaN,0.0,1.0
2560,444,2893,2022-04-29,242,241,444,241,4,NaN,THROWAWAY,NaN,-0.44,96.91,True,False,,NaN,1.0,0.0
2561,445,2893,2022-04-29,242,241,445,241,4,Q4 00:04,REF_TIMEOUT_DISCUSSION???,NaN,NaN,NaN,True,False,,NaN,0.0,0.0
2562,446,2893,2022-04-29,242,241,446,241,4,NaN,D_PENALTY_ON_US,NaN,NaN,NaN,True,False,,NaN,0.0,0.0


In [10]:
df.loc[df['who_turned'] > 0]

,event_index,game_id,date,home_team,away_team,event_counter,team_id,current_quarter,time,event_type,player,x,y,o_point,d_point,line,reciever,turn,who_turned
53,53,2894,2022-05-14,242,245,53,242,1,NaN,POSSESSION,Matthew Mcknight,9.07,49.77,True,False,,NaN,0.0,1.0
60,60,2894,2022-05-14,242,245,60,242,1,NaN,POSSESSION,Dylan Hawkins,23.39,50.65,True,False,,NaN,0.0,1.0
74,74,2894,2022-05-14,242,245,74,242,1,NaN,POSSESSION,Andrew Vande Berg,-9.48,56.86,False,True,,NaN,0.0,1.0
125,125,2894,2022-05-14,242,245,125,242,2,NaN,POSSESSION,Terrence Mitchell,21.75,57.00,True,False,,NaN,0.0,1.0
259,259,2894,2022-05-14,242,245,259,242,3,NaN,POSSESSION,Matthew Mcknight,17.59,50.31,True,False,,NaN,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2396,280,2893,2022-04-29,242,241,280,241,3,NaN,POSSESSION,Khalif El-Salaam,-8.97,73.03,False,True,,NaN,0.0,1.0
2405,289,2893,2022-04-29,242,241,289,241,3,NaN,D_PENALTY_ON_THEM,NaN,NaN,NaN,False,True,,NaN,0.0,1.0
2512,396,2893,2022-04-29,242,241,396,241,4,NaN,O_PENALTY_ON_US,NaN,NaN,NaN,True,False,,NaN,0.0,1.0
2518,402,2893,2022-04-29,242,241,402,241,4,NaN,POSSESSION,Bradley Seuntjens,-1.03,41.50,True,False,,NaN,0.0,1.0


In [11]:
df2 = df.loc[~df['event_type'].isin(pens)].copy()
# df2['turn'] = np.where(df2['event_type'].isin(turnovers), 1, 0)
# df2['who_turned'] = df2['turn'].shift(-1)

In [12]:
df2.loc[(df2['who_turned'] > 0) | (df2['turn'] > 0)]

,event_index,game_id,date,home_team,away_team,event_counter,team_id,current_quarter,time,event_type,player,x,y,o_point,d_point,line,reciever,turn,who_turned
53,53,2894,2022-05-14,242,245,53,242,1,NaN,POSSESSION,Matthew Mcknight,9.07,49.77,True,False,,NaN,0.0,1.0
54,54,2894,2022-05-14,242,245,54,242,1,NaN,THROWAWAY,NaN,-22.72,107.31,True,False,,NaN,1.0,0.0
60,60,2894,2022-05-14,242,245,60,242,1,NaN,POSSESSION,Dylan Hawkins,23.39,50.65,True,False,,NaN,0.0,1.0
61,61,2894,2022-05-14,242,245,61,242,1,NaN,THROWAWAY,NaN,6.61,90.40,True,False,,NaN,1.0,0.0
74,74,2894,2022-05-14,242,245,74,242,1,NaN,POSSESSION,Andrew Vande Berg,-9.48,56.86,False,True,,NaN,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2513,397,2893,2022-04-29,242,241,397,241,4,NaN,THROWAWAY,NaN,-0.22,28.44,True,False,,NaN,1.0,0.0
2518,402,2893,2022-04-29,242,241,402,241,4,NaN,POSSESSION,Bradley Seuntjens,-1.03,41.50,True,False,,NaN,0.0,1.0
2519,403,2893,2022-04-29,242,241,403,241,4,NaN,STALL,NaN,NaN,NaN,True,False,,NaN,1.0,0.0
2559,443,2893,2022-04-29,242,241,443,241,4,NaN,POSSESSION,Khalif El-Salaam,-20.04,37.20,True,False,,NaN,0.0,1.0


In [13]:
df2.loc[df2['event_type'] == "DROP"]

,event_index,game_id,date,home_team,away_team,event_counter,team_id,current_quarter,time,event_type,player,x,y,o_point,d_point,line,reciever,turn,who_turned
402,402,2894,2022-05-14,242,245,402,242,4,NaN,DROP,Alex Davis,14.04,54.81,True,False,,NaN,1.0,0.0
494,53,2894,2022-05-14,242,245,53,245,1,NaN,DROP,Jacob Lane,-12.90,53.11,True,False,,NaN,1.0,0.0
735,294,2894,2022-05-14,242,245,294,245,3,NaN,DROP,Carl Johnson,-10.24,54.20,True,False,,NaN,1.0,0.0
776,335,2894,2022-05-14,242,245,335,245,3,NaN,DROP,Matt McDonald,14.66,39.81,True,False,,Matt McDonald,1.0,0.0
937,45,2905,2022-05-07,245,242,45,245,1,NaN,DROP,Jacob Lane,-22.92,78.00,False,True,,NaN,1.0,0.0
1223,331,2905,2022-05-07,245,242,331,245,3,NaN,DROP,Tannon Hedges,7.70,70.02,False,True,,Tim Lootens,1.0,0.0
1277,385,2905,2022-05-07,245,242,385,245,4,NaN,DROP,Robert Alongi,-8.73,106.15,False,True,,NaN,1.0,0.0
1430,77,2905,2022-05-07,245,242,77,242,2,NaN,DROP,Jacob Fairfax,24.41,107.52,True,False,,NaN,1.0,0.0
1982,270,2893,2022-04-29,242,241,270,242,3,NaN,DROP,Trevor Lynch,2.45,57.47,True,False,,NaN,1.0,0.0


In [14]:
df2.loc[(df2['game_id'] == 2893) & (df2['event_index'] > 265)]

,event_index,game_id,date,home_team,away_team,event_counter,team_id,current_quarter,time,event_type,player,x,y,o_point,d_point,line,reciever,turn,who_turned
1978,266,2893,2022-04-29,242,241,266,242,3,NaN,POSSESSION,Tim McAllister,19.50,42.27,True,False,,NaN,0.0,0.0
1979,267,2893,2022-04-29,242,241,267,242,3,Q3 03:19,OUR_MIDPOINT_TIMEOUT,NaN,NaN,NaN,True,False,,"Jacob Fairfax, Henry Fisher, Matt Gouchoe-Hana...",0.0,0.0
1980,268,2893,2022-04-29,242,241,268,242,3,NaN,SET_O_LINE_NO_PULL,"Jacob Fairfax, Henry Fisher, Matt Gouchoe-Hana...",NaN,NaN,True,False,"[Jacob Fairfax, Henry Fisher, Matt Gouchoe-Han...",Sol Yanuck,0.0,0.0
1981,269,2893,2022-04-29,242,241,269,242,3,NaN,POSSESSION,Sol Yanuck,19.50,42.27,True,False,,Trevor Lynch,0.0,0.5
1982,270,2893,2022-04-29,242,241,270,242,3,NaN,DROP,Trevor Lynch,2.45,57.47,True,False,,NaN,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2558,442,2893,2022-04-29,242,241,442,241,4,NaN,POSSESSION,Austin Taylor,-2.92,26.65,True,False,,Khalif El-Salaam,0.0,0.0
2559,443,2893,2022-04-29,242,241,443,241,4,NaN,POSSESSION,Khalif El-Salaam,-20.04,37.20,True,False,,NaN,0.0,1.0
2560,444,2893,2022-04-29,242,241,444,241,4,NaN,THROWAWAY,NaN,-0.44,96.91,True,False,,NaN,1.0,0.0
2561,445,2893,2022-04-29,242,241,445,241,4,Q4 00:04,REF_TIMEOUT_DISCUSSION???,NaN,NaN,NaN,True,False,,NaN,0.0,0.0
